In [1]:
import os
from dotenv import load_dotenv
import openai
import csv
import pandas as pd
from openai import OpenAI


In [2]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("No OpenAI API key found. Check your .env file.")

In [3]:
# Set the API key for OpenAI
openai.api_key = openai_api_key

In [4]:
reviews = [
    "Absolutely epic! Building this was like reliving my childhood dreams. Five stars!",
    "The detail is amazing, but it took ages to assemble. Worth it though!",
    "Missing pieces in my set. Frustrating experience. Only 2 stars.",
    "Incredible design! My kids and I had a blast building it. Highly recommend.",
    "It’s huge! Takes up too much space. Regret buying.",
    "A true masterpiece for any Star Wars fan. A bit pricey but worth every penny.",
    "The instructions were confusing. Not for the faint-hearted.",
    "Halfway through, and it looks stunning. A challenging yet fun build.",
    "Not for kids, too complex. It's more of an adult Lego set.",
    "Love it! The perfect addition to my Star Wars collection!",
    "Decent build, but I expected more interactive features.",
    "Broke apart after a week. Poor quality for such an expensive set.",
    "A journey through the galaxy! Building it was as epic as the movies.",
    "Overrated. It's just a bunch of bricks at the end of the day.",
    "Magical! Feels like I've built my own Death Star. May the force be with you!",
    "Instructions were great, pieces fit perfectly. A rewarding build.",
    "Expected more minifigures. Kind of disappointed.",
    "Took longer to build than anticipated, but it's a showstopper in my room.",
    "Okay, but there are better Lego sets out there.",
    "Absolutely stunning once assembled, but prepare for a long building journey!"
]



In [11]:
client = OpenAI()
def analyze_sentiments_and_aspect(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=100):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )

    if response.choices and response.choices[0].message:
        content = response.choices[0].message.content
        # Splitting the response into sentiment and aspects parts
        parts = content.split('\n')  # Splitting at the newline character
        sentiment_part = parts[0] if parts else "Sentiment: No sentiment provided"
        aspect_part = parts[1] if len(parts) > 1 else "Aspects: No aspects provided"

        # Extracting sentiment and aspects
        sentiment = sentiment_part.replace('Sentiment:', '').strip()
        aspects = aspect_part.replace('Aspects:', '').strip()

        return sentiment, aspects
    else:
        return "Invalid response", "Invalid response"


In [12]:
# Update the system message to request both sentiment and reasons
delimiter = "####"
system_message = f"""
You will be provided with customer reviews of a Lego set. \
Your task is to classify the sentiment of each review and also collect aspects of what is good or bad, \
Always follow in the following structure: Sentiment: ... Aspects: ... \
The categories of sentiment are: Very positive, slightly positive, neutral, slightly negative, very negative. \
For the apsects you can build your own category. \
The review will be delimited with the following string: {delimiter}
"""

# Initialize an empty list to store review, sentiment, and reasons
data = []

# Analyze each review and store the results
for review in reviews:
    user_message = review
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_message}{delimiter}"}
    ]
    sentiment, aspect = analyze_sentiments_and_aspect(messages)
    data.append({'Review': review, 'Sentiment': sentiment, 'Aspect': aspect})

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

                                               Review          Sentiment  \
0   Absolutely epic! Building this was like relivi...      Very positive   
1   The detail is amazing, but it took ages to ass...      Very positive   
2   Missing pieces in my set. Frustrating experien...      Very negative   
3   Incredible design! My kids and I had a blast b...      Very positive   
4   It’s huge! Takes up too much space. Regret buy...      Very negative   
5   A true masterpiece for any Star Wars fan. A bi...      Very positive   
6   The instructions were confusing. Not for the f...  Slightly negative   
7   Halfway through, and it looks stunning. A chal...      Very positive   
8   Not for kids, too complex. It's more of an adu...  Slightly negative   
9   Love it! The perfect addition to my Star Wars ...      Very positive   
10  Decent build, but I expected more interactive ...  Slightly negative   
11  Broke apart after a week. Poor quality for suc...      Very negative   
12  A journe

In [13]:
df.to_csv('sentiment_analysis_results4.csv', index=False)